In [1]:
import pytesseract
import cv2
import re
from pytesseract import Output
import numpy as np
from dateutil import parser

In [11]:
image = cv2.imread('aadhar.jpg') #to read the image provided by the user
count=0  #I make a count variable to count the number of checkpoints 


## Checkpoint - 1: Face Detection in Aadhar Card using OpenCV
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
faces = face_cascade.detectMultiScale(gray, 1.3, 7)
try:
    if(faces).all():
        print("Face Detected - Checkpoint 1")
        count+=1
except:
    print("No Face Detected")

for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    roi_color = image[y:y + h, x:x + w]
    crop_pic = cv2.imshow('croppicds', roi_color)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## Code to remove unwanted noises from Image using OpenCV
rgb_planes = cv2.split(image)
result_planes = []
result_norm_planes = []
for plane in rgb_planes:
    dilated_img = cv2.dilate(plane, np.ones((10, 10), np.uint8))    
    bg_img = cv2.medianBlur(dilated_img, 21)
    diff_img = 255 - cv2.absdiff(plane, bg_img)
    norm_img = cv2.normalize(diff_img, None, alpha=0, beta=250, norm_type=cv2.NORM_MINMAX,dtype=cv2.CV_8UC1)
    result_planes.append(diff_img)
    result_norm_planes.append(norm_img)

result = cv2.merge(result_planes)
result_norm = cv2.merge(result_norm_planes)
dst = cv2.fastNlMeansDenoisingColored(result_norm, None, 10, 10, 7, 11)   
text = pytesseract.image_to_string(dst).upper().replace(" ", "")


##Checkpoint -2: Gender Detection and Validation
sex = str(re.findall(r"(MALE|FEMALE)", text)).replace("[","").replace("'", "").replace("]", "")
print(sex)

try:
    print("Sex confirmed - Checkpoint 2")
    count+=1
except:
    print("Invalid Sex")

## Checkpoint -3: Date Of Birth Detection and Validation
date = str(re.findall(r"[\d]{1,4}[/-][\d]{1,4}[/-][\d]{1,4}", text)).replace("]", "").replace("[","").replace("'", "")
print(date)
try:
    print(bool(parser.parse(date)),"Date - Checkpoint 3")
    count+=1
except ValueError:
    print("Incorrect date format")

## Checkpoint -4: Aadhar Number Detection and Validation
aadhar_number = str(re.findall(r"[0-9]{11,12}", text)).replace("]", "").replace("[","").replace("'", "")
print(aadhar_number)
if len(aadhar_number)<12 or len(aadhar_number)>12:
    print("Invalid Aadhar Number")
else:
    if aadhar_number[0]=="0" or aadhar_number[0]=="1":
        print("Invalid Aadhar Number")
    else:
        try:
            print("Valid Aadhar Number - Checkpoint 4")
            count+=1
        except:
            print("Invalid")

## Finally, it will check all the checkpoints and confirmed whether the given Image is a VALID Aadhar Card or not
if(count==4):
    print('.....The given Aadhar Card is Valid.....')
else:
    print(".....This is an Invalid Aadhar Card.....")

Face Detected - Checkpoint 1
MALE
Sex confirmed - Checkpoint 2
20/03/2002
True Date - Checkpoint 3
305610425728
Valid Aadhar Number - Checkpoint 4
.....The given Aadhar Card is Valid.....
